## Import Libraries

In [4]:
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from xgboost import XGBClassifier
import cv2
import numpy as np
from scipy.ndimage import sobel, generic_filter
from skimage.filters.rank import entropy
from skimage.morphology import disk
import random
import shutil
from sklearn.metrics import classification_report, f1_score, confusion_matrix, accuracy_score, precision_score, recall_score, jaccard_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
from PIL import Image
import matplotlib.pyplot as plt
# from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
# import segmentation_models_pytorch as smp

## Data preprocessing (for image segmentation and machine learning)

In [5]:
# Load and sort image file names
mask_path = './data/masks/'
nrg_path = './data/NRG_images/'
grb_path = './data/RGB_images/'

mask_files = sorted([f for f in os.listdir(mask_path) if f.endswith('.png')])
nrg_files = sorted([f for f in os.listdir(nrg_path) if f.endswith('.png')])
grb_files = sorted([f for f in os.listdir(grb_path) if f.endswith('.png')])

# Combine file names into tuples and split into train/test sets
all_files = list(zip(mask_files, nrg_files, grb_files))
train_files, test_files = train_test_split(all_files, test_size=0.2, random_state=42)

In [6]:
# Load and resize image
def load_and_resize(image_path, size=(128, 128), is_mask=False):
    if is_mask:
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        resized = cv2.resize(img, size, interpolation=cv2.INTER_NEAREST)
    else:
        img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        resized = cv2.resize(img, size, interpolation=cv2.INTER_AREA)
    return resized

# Get mean and variance
def getMeanAndVar(channel, window = 5):
    mean = cv2.blur(channel, (window, window))
    var = generic_filter(channel, np.var, size = window)
    return mean, var

# Extract features from one image
def extractFeatures(mask_img, nrg_img, rgb_img, window = 5):
    h, w, _ = rgb_img.shape
    nrg = nrg_img[:, :, 0].astype(np.float32)
    r = rgb_img[:, :, 2].astype(np.float32)
    g = rgb_img[:, :, 1].astype(np.float32)
    b = rgb_img[:, :, 0].astype(np.float32)
    ndvi = (nrg - r) / (nrg + r + 1e-6)
    gndvi = (nrg - g) / (nrg + g + 1e-6)
    
    # Sobel strength
    sobel_x = sobel(ndvi, axis=0)
    sobel_y = sobel(ndvi, axis=1)
    ndvi_sobel = np.hypot(sobel_x, sobel_y)

    # Local entropy: texture chaos
    gray = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2GRAY).astype(np.uint8)
    entropy_img = entropy(gray, disk(3))
    hsv = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2HSV)
    saturation = hsv[:, :, 1].astype(np.float32)
    lab = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2LAB)
    l = lab[:, :, 0].astype(np.float32)

    # Get mean and variance of each chanel
    r_mean, r_var = getMeanAndVar(r, window)
    g_mean, g_var = getMeanAndVar(g, window)
    b_mean, b_var = getMeanAndVar(b, window)
    nrg_mean, nrg_var = getMeanAndVar(nrg, window)
    ndvi_mean, ndvi_var = getMeanAndVar(ndvi, window)

     # Get coordinate information
    x_coords, y_coords = np.meshgrid(np.arange(w), np.arange(h))
    x_norm = (x_coords / w).astype(np.float32)
    y_norm = (y_coords / h).astype(np.float32)

    # Concatenate all features
    X = np.stack([
        r.reshape(-1), g.reshape(-1), b.reshape(-1), nrg.reshape(-1),
        r_mean.reshape(-1), g_mean.reshape(-1), b_mean.reshape(-1), nrg_mean.reshape(-1),
        r_var.reshape(-1), g_var.reshape(-1), b_var.reshape(-1), nrg_var.reshape(-1), ndvi.reshape(-1), 
        ndvi_mean.reshape(-1), ndvi_var.reshape(-1), x_norm.reshape(-1), y_norm.reshape(-1),
        gndvi.reshape(-1), ndvi_sobel.reshape(-1), entropy_img.reshape(-1), saturation.reshape(-1), l.reshape(-1)
    ], axis=1)

    y = (mask_img > 127).astype(np.uint8).reshape(-1)

    return X, y

In [7]:
# Build training set features
X_train, y_train = [], []
for mask_name, nrg_name, rgb_name in train_files:
    mask = load_and_resize(os.path.join(mask_path, mask_name), is_mask=True)
    nrg = load_and_resize(os.path.join(nrg_path, nrg_name))
    rgb = load_and_resize(os.path.join(grb_path, rgb_name))
    feats, labels = extractFeatures(mask, nrg, rgb)
    X_train.append(feats)
    y_train.append(labels)
X_train = np.vstack(X_train)
y_train = np.hstack(y_train)

# Build test set features
X_test, y_test = [], []
for mask_name, nrg_name, rgb_name in test_files:
    mask = load_and_resize(os.path.join(mask_path, mask_name), is_mask=True)
    nrg = load_and_resize(os.path.join(nrg_path, nrg_name))
    rgb = load_and_resize(os.path.join(grb_path, rgb_name))
    feats, labels = extractFeatures(mask, nrg, rgb)
    X_test.append(feats)
    y_test.append(labels)
X_test = np.vstack(X_test)
y_test = np.hstack(y_test)

## Image Segmentation

In [44]:
# Traditional method: image segmentation
X = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)

entroy_img = X[:,19]
ndvi = X[:,12]

pred_mask = ((entroy_img > 0.9) & (ndvi < 0)).astype(np.uint8)

# Report various indicators
print('-----------Image Segmentation----------------')
print(f"Accuracy: {accuracy_score(y, pred_mask):.4f}")
print(f'Precision: {precision_score(y, pred_mask):.4f}')
print(f'Recall: {recall_score(y, pred_mask):.4f}')
print(f'F1 Score: {f1_score(y, pred_mask):.4f}')
print(f"IOU Score: {jaccard_score(y, pred_mask):.4f}")
print("Confusion Matrix:\n", confusion_matrix(y, pred_mask))
print("Classification Report:\n", classification_report(y, pred_mask))

-----------Image Segmentation----------------
Accuracy: 0.8085
Precision: 0.0547
Recall: 0.5610
F1 Score: 0.0996
IOU Score: 0.0524
Confusion Matrix:
 [[5803987 1333115]
 [  60310   77084]]
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.81      0.89   7137102
           1       0.05      0.56      0.10    137394

    accuracy                           0.81   7274496
   macro avg       0.52      0.69      0.50   7274496
weighted avg       0.97      0.81      0.88   7274496



## Machine Learning

In [45]:
# SVM & XGBoost Voting Model

# Resample the training set
def sample_balanced(X, y, n_pos, n_neg, random_state=42):
    np.random.seed(random_state)

    pos_indices = np.where(y == 1)[0]
    neg_indices = np.where(y == 0)[0]

    pos_sample = np.random.choice(pos_indices, size=min(n_pos, len(pos_indices)), replace=False)
    neg_sample = np.random.choice(neg_indices, size=min(n_neg, len(neg_indices)), replace=False)

    selected_indices = np.concatenate([pos_sample, neg_sample])
    np.random.shuffle(selected_indices)

    return X[selected_indices], y[selected_indices]

X_small, y_small = sample_balanced(X_train, y_train, n_pos=10000, n_neg=10000)

# Choose suitable features for each model
svm_features = [12, 13, 14, 19, 20]
xgb_features = [12, 13, 14, 17, 18, 19, 20, 21]
X_train_svm = X_small[:, svm_features]
X_test_svm = X_test[:, svm_features]
X_train_xgb = X_small[:, xgb_features]
X_test_xgb = X_test[:, xgb_features]

# SVM model is not suitable for training with too much data
X_train_svm, y_train_svm = sample_balanced(X_train_svm, y_small, n_pos=5000, n_neg=2000)

# Define and train SVM classifier
svm_model = SVC(
    C=10,
    gamma=0.01,
    kernel='rbf',
    class_weight='balanced',
    probability=True,
)
svm_model.fit(X_train_svm, y_train_svm)

# Define and train XGBoost classifier
xgb_clf = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    max_depth=3,
    learning_rate=0.05,
    n_estimators=1000,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=1,
    reg_alpha=5,
    reg_lambda=10,
    gamma=1,
    use_label_encoder=False,
    verbosity=0,
    random_state=42
)

xgb_clf.fit(
    X_train_xgb, y_small,
    eval_set=[(X_test_xgb, y_test)],
    verbose=False
)

# Predict
proba_svm = svm_model.predict_proba(X_test_svm)[:, 1]
proba_xgb = xgb_clf.predict_proba(X_test_xgb)[:, 1]
avg_proba = 0.12 * proba_svm + 0.88 * proba_xgb
best_preds = (avg_proba > 0.863).astype(int)

# Report various indicators
print('-----------Voting Model----------------')
print(f"Accuracy: {accuracy_score(y_test, best_preds):.4f}")
print(f'Precision: {precision_score(y_test, best_preds):.4f}')
print(f'Recall: {recall_score(y_test, best_preds):.4f}')
print(f'F1 Score: {f1_score(y_test, best_preds):.4f}')
print(f"IOU Score: {jaccard_score(y_test, best_preds):.4f}")
print("Confusion Matrix:\n", confusion_matrix(y_test, best_preds))
print("Classification Report:\n", classification_report(y_test, best_preds))

-----------Voting Model----------------
Accuracy: 0.9668
Precision: 0.2325
Recall: 0.3181
F1 Score: 0.2686
IOU Score: 0.1552
Confusion Matrix:
 [[1400842   29372]
 [  19066    8896]]
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.98      0.98   1430214
           1       0.23      0.32      0.27     27962

    accuracy                           0.97   1458176
   macro avg       0.61      0.65      0.63   1458176
weighted avg       0.97      0.97      0.97   1458176



In [8]:
# Random Forest

# Choose suitable features for the model
rf_features = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
X_train_rf = X_train[:, rf_features]
X_test_rf = X_test[:, rf_features]

# Define and train Random Forest classifier
random_tree_model = RandomForestClassifier(
    n_estimators = 50,
    random_state = 10,
    max_depth = 15,
    n_jobs = -1,
    class_weight = 'balanced')
random_tree_model.fit(X_train_rf, y_train)
proba_forest = random_tree_model.predict_proba(X_test_rf)[:, 1]
forest_pred = (proba_forest > 0.748).astype(int)

# Report various indicators
print('-----------Random Forest----------------')
print(f"Accuracy: {accuracy_score(y_test, forest_pred):.4f}")
print(f'Precision: {precision_score(y_test, forest_pred):.4f}')
print(f'Recall: {recall_score(y_test, forest_pred):.4f}')
print(f'F1 Score: {f1_score(y_test, forest_pred):.4f}')
print(f"IOU Score: {jaccard_score(y_test, forest_pred):.4f}")
print("Confusion Matrix:\n", confusion_matrix(y_test, forest_pred))
print("Classification Report:\n", classification_report(y_test, forest_pred))

-----------Random Forest----------------
Accuracy: 0.9636
Precision: 0.2500
Recall: 0.4502
F1 Score: 0.3215
IOU Score: 0.1916
Confusion Matrix:
 [[1392460   37754]
 [  15374   12588]]
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.97      0.98   1430214
           1       0.25      0.45      0.32     27962

    accuracy                           0.96   1458176
   macro avg       0.62      0.71      0.65   1458176
weighted avg       0.97      0.96      0.97   1458176



## Deep learning

In [10]:
class DeadTreeDataset(Dataset):
    def __init__(self, rgb_paths, nir_paths, mask_paths, transform=None, img_size=128):
        self.rgb_paths = rgb_paths
        self.nir_paths = nir_paths
        self.mask_paths = mask_paths
        self.transform = transform
        self.img_size = img_size

    def __len__(self):
        return len(self.rgb_paths)

    def __getitem__(self, idx):
        while True:
            rgb = Image.open(self.rgb_paths[idx]).convert("RGB")
            nir = Image.open(self.nir_paths[idx]).convert("L")
            mask = Image.open(self.mask_paths[idx]).convert("L")

            rgb = rgb.resize((self.img_size, self.img_size))
            nir = nir.resize((self.img_size, self.img_size))
            mask = mask.resize((self.img_size, self.img_size), Image.NEAREST)

            rgb = np.array(rgb)
            nir = np.expand_dims(np.array(nir), axis=2)
            mask_arr = np.array(mask).astype(np.float32) / 255.0
            if mask_arr.max() == 255.0:
                mask_arr /= 255.0
            if mask_arr.max() == 0:
                idx = (idx + 1) % len(self.rgb_paths)
                continue

            image = np.concatenate([rgb, nir], axis=2)

            if self.transform:
                image, mask_arr = self.transform(image, mask_arr)

            image = torch.from_numpy(image.transpose((2, 0, 1))).float()
            mask = torch.from_numpy(mask_arr).unsqueeze(0).float()

            return image, mask

In [11]:
class AdvancedTransforms4Channel:
    def __init__(self, img_size=128):
        self.img_size = img_size
        self.mean = np.array([0.485, 0.456, 0.406, 0.5])
        self.std = np.array([0.229, 0.224, 0.225, 0.5])
        self.train = self._train_transform
        self.val = self._val_transform

    def _train_transform(self, image, mask):
        if np.random.rand() > 0.5:
            image = np.flip(image, axis=1)
            mask = np.flip(mask, axis=1)
        if np.random.rand() > 0.5:
            image = np.flip(image, axis=0)
            mask = np.flip(mask, axis=0)
        if np.random.rand() > 0.5:
            angle = np.random.randint(-15, 15)
            M = cv2.getRotationMatrix2D((self.img_size//2, self.img_size//2), angle, 1)
            image = cv2.warpAffine(image, M, (self.img_size, self.img_size), borderMode=cv2.BORDER_REFLECT)
            mask = cv2.warpAffine(mask, M, (self.img_size, self.img_size), flags=cv2.INTER_NEAREST)
        return self._normalize(image), mask.astype(np.float32).copy()


    def _val_transform(self, image, mask):
        return self._normalize(image), mask.copy()

    def _normalize(self, image):
        image = image / 255.0
        return (image - self.mean) / self.std

In [12]:
### Unet
class DownBlock(nn.Module):
    """
    Encoder: Conv3x3 + ReLU -> Conv3x3 + ReLU -> MaxPool2d
    """
    def __init__(self, in_channels, out_channels):
        super(DownBlock, self).__init__()
        self.Conv_BN_ReLU_2 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, stride=1,padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )
        self.downpool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self,x):
        conv_out=self.Conv_BN_ReLU_2(x)
        pooled_out=self.downpool(conv_out)
        return conv_out, pooled_out

class UpBlock(nn.Module):
    """
    Decoder：Conv3x3 + ReLU -> Conv3x3 + ReLU -> UpConv2x2
    """
    def __init__(self, in_channels, out_channels):
        super(UpBlock, self).__init__()
        self.Conv_BN_ReLU_2 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels*2, kernel_size=3, stride=1,padding=1),
            nn.BatchNorm2d(out_channels*2),
            nn.ReLU(),
            nn.Conv2d(in_channels=out_channels*2, out_channels=out_channels*2, kernel_size=3, stride=1,padding=1),
            nn.BatchNorm2d(out_channels*2),
            nn.ReLU()
        )
        self.upconv = nn.Sequential(
            nn.ConvTranspose2d(in_channels=out_channels*2,out_channels=out_channels,kernel_size=3,stride=2,padding=1,output_padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

    def forward(self,x,out):
        x_out=self.Conv_BN_ReLU_2(x)
        x_out=self.upconv(x_out)
        cat_out=torch.cat((x_out,out),dim=1)
        return cat_out

class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super(UNet, self).__init__()
        out = [64, 128, 256, 512, 1024]
        # Encoder
        self.down1 = DownBlock(in_channels, out[0])      # 3-64
        self.down2 = DownBlock(out[0], out[1])           # 64-128
        self.down3 = DownBlock(out[1], out[2])           # 128-256
        self.down4 = DownBlock(out[2], out[3])           # 256-512

        # Decoder
        self.up1 = UpBlock(out[3], out[3])           # 512-1024-512*2
        self.up2 = UpBlock(out[4], out[2])           # 1024-512-256*2
        self.up3 = UpBlock(out[3], out[1])           # 512-256-128*2
        self.up4 = UpBlock(out[2], out[0])           # 256-128-64*2

        # Output
        self.output = nn.Sequential(
            nn.Conv2d(out[1],out[0],kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(out[0]),
            nn.ReLU(),
            nn.Conv2d(out[0],out[0],kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out[0]),
            nn.ReLU(),
            nn.Conv2d(out[0],out_channels,kernel_size=3,stride=1,padding=1)
        )

    def forward(self, x):
        # Encoder
        x1, x = self.down1(x)   # 3-64
        x2, x = self.down2(x)   # 64-128
        x3, x = self.down3(x)   # 128-256
        x4, x = self.down4(x)   # 256-512

        # Decoder
        x5 = self.up1(x, x4)      # 512-512
        x6 = self.up2(x5, x3)     # 512-256
        x7 = self.up3(x6, x2)     # 256-128
        x8 = self.up4(x7, x1)     # 128-64

        out = self.output(x8)    # 128-64-64-1
        return out

In [13]:
### ECA UNet
class ECABlock(nn.Module):
    """Constructs a ECA module.

    Args:
        channel: Number of channels of the input feature map
        k_size: Adaptive selection of kernel size
    """
    def __init__(self, channel, k_size=3):
        super(ECABlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.conv = nn.Conv1d(1, 1, kernel_size=k_size, padding=(k_size - 1) // 2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # feature descriptor on the global spatial information
        y = self.avg_pool(x)

        # Two different branches of ECA module
        y = self.conv(y.squeeze(-1).transpose(-1, -2)).transpose(-1, -2).unsqueeze(-1)

        # Multi-scale information fusion
        y = self.sigmoid(y)

        return x * y.expand_as(x)

class ECADownBlock(nn.Module):
    """
    Encoder
    """
    def __init__(self, in_channels, out_channels):
        super(ECADownBlock, self).__init__()
        self.Conv_BN_ReLU_2 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, stride=1,padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )
        self.downpool = nn.Sequential(
            nn.Conv2d(in_channels=out_channels,out_channels=out_channels,kernel_size=3,stride=2,padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )
        self.eca = ECABlock(out_channels)

    def forward(self,x):
        conv_out=self.Conv_BN_ReLU_2(x)
        conv_out = self.eca(conv_out)
        pooled_out=self.downpool(conv_out)
        return conv_out, pooled_out

class ECAUpBlock(nn.Module):
    """
    Decoder
    """
    def __init__(self, in_channels, out_channels):
        # 512-1024-512
        # 1024-512-256
        # 512-256-128
        # 256-128-64
        super(ECAUpBlock, self).__init__()
        self.Conv_BN_ReLU_2 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels*2, kernel_size=3, stride=1,padding=1),
            nn.BatchNorm2d(out_channels*2),
            nn.ReLU(),
            nn.Conv2d(in_channels=out_channels*2, out_channels=out_channels*2, kernel_size=3, stride=1,padding=1),
            nn.BatchNorm2d(out_channels*2),
            nn.ReLU()
        )
        self.upconv = nn.Sequential(
            nn.ConvTranspose2d(in_channels=out_channels*2,out_channels=out_channels,kernel_size=3,stride=2,padding=1,output_padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )
        self.eca = ECABlock(out_channels * 2)

    def forward(self,x,out):
        x_out=self.Conv_BN_ReLU_2(x)
        x_out= self.eca(x_out)
        x_out=self.upconv(x_out)
        cat_out=torch.cat((x_out,out),dim=1)
        return cat_out

class ECAUNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super(ECAUNet, self).__init__()
        out = [64, 128, 256, 512, 1024]
        # Encoder
        self.down1 = ECADownBlock(in_channels, out[0])      # 3-64
        self.down2 = ECADownBlock(out[0], out[1])           # 64-128
        self.down3 = ECADownBlock(out[1], out[2])           # 128-256
        self.down4 = ECADownBlock(out[2], out[3])           # 256-512

        # Decoder
        self.up1 = ECAUpBlock(out[3], out[3])           # 512-1024-512*2
        self.up2 = ECAUpBlock(out[4], out[2])           # 1024-512-256*2
        self.up3 = ECAUpBlock(out[3], out[1])           # 512-256-128*2
        self.up4 = ECAUpBlock(out[2], out[0])           # 256-128-64*2

        # Output
        self.output = nn.Sequential(
            nn.Conv2d(out[1],out[0],kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(out[0]),
            nn.ReLU(),
            nn.Conv2d(out[0],out[0],kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out[0]),
            nn.ReLU(),
            nn.Conv2d(out[0],out_channels,kernel_size=3,stride=1,padding=1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x1, x = self.down1(x)   # 3-64
        x2, x = self.down2(x)   # 64-128
        x3, x = self.down3(x)   # 128-256
        x4, x = self.down4(x)   # 256-512
        x5 = self.up1(x, x4)      # 512-512
        x6 = self.up2(x5, x3)     # 512-256
        x7 = self.up3(x6, x2)     # 256-128
        x8 = self.up4(x7, x1)     # 128-64
        out = self.output(x8)    # 128-64-64-1
        return out

In [15]:
mask_files = sorted([os.path.join(mask_path, f) for f in os.listdir(mask_path) if f.endswith('.png')])
rgb_files = sorted([os.path.join(grb_path, f) for f in os.listdir(grb_path) if f.endswith('.png')])
nir_files = sorted([os.path.join(nrg_path, f) for f in os.listdir(nrg_path) if f.endswith('.png')])

all_data = list(zip(rgb_files, nir_files, mask_files))
train_data, val_data = train_test_split(all_data, test_size=0.2, random_state=42)

advanced_tf = AdvancedTransforms4Channel(img_size=128)

train_dataset = DeadTreeDataset(
    [x[0] for x in train_data], [x[1] for x in train_data], [x[2] for x in train_data],
    transform=advanced_tf.train
)

val_dataset = DeadTreeDataset(
    [x[0] for x in val_data], [x[1] for x in val_data], [x[2] for x in val_data],
    transform=advanced_tf.val
)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
class IoULoss(nn.Module):
    def __init__(self, smooth=1e-6):
        super(IoULoss, self).__init__()
        self.smooth = smooth

    def forward(self, preds, targets):
        preds = preds.view(-1)
        targets = targets.view(-1)
        intersection = (preds * targets).sum()
        union = preds.sum() + targets.sum() - intersection
        iou = (intersection + self.smooth) / (union + self.smooth)
        return 1 - iou

class ComboLoss(nn.Module):
    def __init__(self, alpha=0.5):
        super(ComboLoss, self).__init__()
        self.bce = nn.BCELoss()
        self.iou = IoULoss()
        self.alpha = alpha

    def forward(self, preds, targets):
        return self.alpha * self.bce(preds, targets) + (1 - self.alpha) * self.iou(preds, targets)

In [17]:
criterion1 = nn.BCEWithLogitsLoss()
criterion2 = nn.BCELoss()
criterion3 = ComboLoss(alpha=0.6)

Resnet50 = smp.Unet(
    encoder_name="resnet50",
    encoder_weights="imagenet",
    in_channels=4,
    classes=1
).to(device)
unet = UNet(in_channels=4, out_channels=1).to(device)
ECAunet = ECAUNet(in_channels=4, out_channels=1).to(device)

config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

c:\Users\11970\anaconda3\envs\gpu\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\11970\.cache\huggingface\hub\models--smp-hub--resnet50.imagenet. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

In [18]:
def compute_iou(preds, masks, threshold):
    preds = (preds > threshold).int().cpu().numpy().flatten()
    masks = masks.int().cpu().numpy().flatten()
    iou = jaccard_score(masks, preds, average='binary', zero_division=0)
    return iou

def train_model(model, train_loader, val_loader, criterion, device, num_epochs, patience=10):
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    best_val_loss = float("inf")
    counter = 0
    best_thresh = 0.5

    save_dir = "checkpoints"
    os.makedirs(save_dir, exist_ok=True)

    train_loss_list = []
    val_loss_list = []
    iou_list = []

    final_preds = None
    final_masks = None

    for epoch in range(1, num_epochs + 1):
        # === Train ===
        model.train()
        train_loss = 0
        for images, masks in tqdm(train_loader, desc=f"[Epoch {epoch}] Training"):
            images, masks = images.to(device), masks.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            if isinstance(criterion, nn.BCELoss) or criterion.__class__.__name__ == "ComboLoss":
                preds_prob = torch.sigmoid(outputs)
                loss = criterion(preds_prob, masks.float())
            else:
                loss = criterion(outputs, masks.float())
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
        train_loss /= len(train_loader)

        # === Validation ===
        model.eval()
        val_loss, all_preds, all_masks = 0, [], []
        with torch.no_grad():
            for images, masks in val_loader:
                images, masks = images.to(device), masks.to(device)
                outputs = model(images)

                if isinstance(criterion, nn.BCELoss) or criterion.__class__.__name__ == "ComboLoss":
                    preds_prob = torch.sigmoid(outputs)
                    loss = criterion(preds_prob, masks.float())
                    all_preds.append(preds_prob)
                else:
                    loss = criterion(outputs, masks.float())
                    all_preds.append(torch.sigmoid(outputs))

                val_loss += loss.item()
                all_masks.append(masks)

        val_loss /= len(val_loader)
        all_preds = torch.cat(all_preds, dim=0)
        all_masks = torch.cat(all_masks, dim=0)

        best_iou = 0
        for t in np.arange(0.3, 0.7, 0.05):
            iou = compute_iou(all_preds, all_masks, threshold=t)
            if iou > best_iou:
                best_iou = iou
                best_thresh = t
                final_preds = all_preds.clone()
                final_masks = all_masks.clone()

        train_loss_list.append(train_loss)
        val_loss_list.append(val_loss)
        iou_list.append(best_iou)

        print(f"[Epoch {epoch}] Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, IoU: {best_iou:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping triggered.")
                break

    return {
        "train_loss": train_loss_list,
        "val_loss": val_loss_list,
        "iou": iou_list,
        "best_thresh": best_thresh,
        "all_preds": final_preds,
        "all_masks": final_masks
    }

def evaluate_metrics(results):
    best_thresh = results['best_thresh']
    all_preds = results['all_preds']
    all_masks = results['all_masks']

    preds_bin = (all_preds > best_thresh).int().cpu().numpy().flatten()
    masks_bin = all_masks.int().cpu().numpy().flatten()

    preds_bin = np.round(preds_bin).astype(int)
    masks_bin = np.round(masks_bin).astype(int)

    acc = accuracy_score(masks_bin, preds_bin)
    prec = precision_score(masks_bin, preds_bin, zero_division=0)
    rec = recall_score(masks_bin, preds_bin, zero_division=0)
    f1 = f1_score(masks_bin, preds_bin, zero_division=0)
    cm = confusion_matrix(masks_bin, preds_bin)

    print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1 Score: {f1:.4f}")
    print("Confusion Matrix:")
    print(cm)

    return {
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1,
        "confusion_matrix": cm
    }

In [19]:
results_res = train_model(Resnet50, train_loader, val_loader, criterion1, device, num_epochs=80)
metrics = evaluate_metrics(results_res)

[Epoch 1] Training: 100%|██████████| 45/45 [00:09<00:00,  4.54it/s]


[Epoch 1] Train Loss: 0.6148, Val Loss: 0.4890, IoU: 0.0231


[Epoch 2] Training: 100%|██████████| 45/45 [00:06<00:00,  7.25it/s]


[Epoch 2] Train Loss: 0.4259, Val Loss: 0.3544, IoU: 0.0242


[Epoch 3] Training: 100%|██████████| 45/45 [00:06<00:00,  7.37it/s]


...

[Epoch 78] Train Loss: 0.0390, Val Loss: 0.0467, IoU: 0.3487


[Epoch 79] Training: 100%|██████████| 45/45 [00:06<00:00,  7.50it/s]


[Epoch 79] Train Loss: 0.0388, Val Loss: 0.0469, IoU: 0.3442


[Epoch 80] Training: 100%|██████████| 45/45 [00:06<00:00,  7.46it/s]


[Epoch 80] Train Loss: 0.0374, Val Loss: 0.0470, IoU: 0.3573
Accuracy: 0.9799, Precision: 0.4829, Recall: 0.5789, F1 Score: 0.5265
Confusion Matrix:
[[1412579   17449]
 [  11854   16294]]


In [20]:
unet = UNet(in_channels=4, out_channels=1).to(device)
results_unet_BCEWithLogitsLoss = train_model(unet, train_loader, val_loader, criterion1, device, num_epochs=80)
metrics = evaluate_metrics(results_unet_BCEWithLogitsLoss)

[Epoch 1] Training: 100%|██████████| 45/45 [00:09<00:00,  4.83it/s]


[Epoch 1] Train Loss: 0.3821, Val Loss: 0.2403, IoU: 0.0884


[Epoch 2] Training: 100%|██████████| 45/45 [00:09<00:00,  4.96it/s]


[Epoch 2] Train Loss: 0.1856, Val Loss: 0.1547, IoU: 0.1700


[Epoch 3] Training: 100%|██████████| 45/45 [00:09<00:00,  4.97it/s]


...

[Epoch 60] Train Loss: 0.0353, Val Loss: 0.0400, IoU: 0.3948


[Epoch 61] Training: 100%|██████████| 45/45 [00:09<00:00,  4.89it/s]


[Epoch 61] Train Loss: 0.0349, Val Loss: 0.0413, IoU: 0.4026


[Epoch 62] Training: 100%|██████████| 45/45 [00:09<00:00,  4.88it/s]


[Epoch 62] Train Loss: 0.0349, Val Loss: 0.0409, IoU: 0.4033
Early stopping triggered.
Accuracy: 0.9834, Precision: 0.5695, Recall: 0.5803, F1 Score: 0.5748
Confusion Matrix:
[[1417682   12346]
 [  11815   16333]]


In [21]:
# BCELoss
unet = UNet(in_channels=4, out_channels=1).to(device)
results_unet_BCELoss = train_model(unet, train_loader, val_loader, criterion2, device, num_epochs=80)
metrics = evaluate_metrics(results_unet_BCELoss)

[Epoch 1] Training: 100%|██████████| 45/45 [00:09<00:00,  4.88it/s]


[Epoch 1] Train Loss: 0.4693, Val Loss: 0.3169, IoU: 0.0528


[Epoch 2] Training: 100%|██████████| 45/45 [00:09<00:00,  4.88it/s]


[Epoch 2] Train Loss: 0.2391, Val Loss: 0.1973, IoU: 0.1157


[Epoch 3] Training: 100%|██████████| 45/45 [00:09<00:00,  4.88it/s]


...

[Epoch 67] Train Loss: 0.0375, Val Loss: 0.0398, IoU: 0.4186


[Epoch 68] Training: 100%|██████████| 45/45 [00:09<00:00,  4.92it/s]


[Epoch 68] Train Loss: 0.0367, Val Loss: 0.0389, IoU: 0.4215


[Epoch 69] Training: 100%|██████████| 45/45 [00:09<00:00,  4.92it/s]


[Epoch 69] Train Loss: 0.0361, Val Loss: 0.0395, IoU: 0.4187
Early stopping triggered.
Accuracy: 0.9842, Precision: 0.5892, Recall: 0.5913, F1 Score: 0.5903
Confusion Matrix:
[[1418423   11605]
 [  11503   16645]]


In [22]:
# ComboLoss
unet = UNet(in_channels=4, out_channels=1).to(device)
results_unet_comboLoss = train_model(unet, train_loader, val_loader, criterion3, device, num_epochs=80)
metrics = evaluate_metrics(results_unet_comboLoss)

[Epoch 1] Training: 100%|██████████| 45/45 [00:09<00:00,  4.85it/s]


[Epoch 1] Train Loss: 0.6131, Val Loss: 0.4923, IoU: 0.1477


[Epoch 2] Training: 100%|██████████| 45/45 [00:09<00:00,  4.94it/s]


[Epoch 2] Train Loss: 0.4800, Val Loss: 0.4481, IoU: 0.2664


[Epoch 3] Training: 100%|██████████| 45/45 [00:09<00:00,  4.93it/s]


...

[Epoch 65] Train Loss: 0.2559, Val Loss: 0.2818, IoU: 0.3965


[Epoch 66] Training: 100%|██████████| 45/45 [00:09<00:00,  4.92it/s]


[Epoch 66] Train Loss: 0.2541, Val Loss: 0.2750, IoU: 0.4129


[Epoch 67] Training: 100%|██████████| 45/45 [00:09<00:00,  4.91it/s]


[Epoch 67] Train Loss: 0.2567, Val Loss: 0.2725, IoU: 0.4242
Early stopping triggered.
Accuracy: 0.9842, Precision: 0.5903, Recall: 0.6012, F1 Score: 0.5957
Confusion Matrix:
[[1418283   11745]
 [  11226   16922]]


In [23]:
# BCEWithLogitsLoss
ECAunet = ECAUNet(in_channels=4, out_channels=1).to(device)
results_ECA_BCEWithLogitsLoss = train_model(ECAunet, train_loader, val_loader, criterion1, device, num_epochs=80)
metrics = evaluate_metrics(results_ECA_BCEWithLogitsLoss)

[Epoch 1] Training: 100%|██████████| 45/45 [00:09<00:00,  4.59it/s]


[Epoch 1] Train Loss: 0.8507, Val Loss: 0.7856, IoU: 0.0489


[Epoch 2] Training: 100%|██████████| 45/45 [00:09<00:00,  4.68it/s]


[Epoch 2] Train Loss: 0.7574, Val Loss: 0.7382, IoU: 0.0193


[Epoch 3] Training: 100%|██████████| 45/45 [00:09<00:00,  4.67it/s]


...

[Epoch 78] Train Loss: 0.6919, Val Loss: 0.6922, IoU: 0.2743


[Epoch 79] Training: 100%|██████████| 45/45 [00:09<00:00,  4.66it/s]


[Epoch 79] Train Loss: 0.6918, Val Loss: 0.6921, IoU: 0.3140


[Epoch 80] Training: 100%|██████████| 45/45 [00:09<00:00,  4.69it/s]


[Epoch 80] Train Loss: 0.6919, Val Loss: 0.6923, IoU: 0.2388
Accuracy: 0.9840, Precision: 0.7410, Recall: 0.2606, F1 Score: 0.3855
Confusion Matrix:
[[1427465    2563]
 [  20814    7334]]


In [24]:
# BCELoss
ECAunet = ECAUNet(in_channels=4, out_channels=1).to(device)
results_ECA_BCELoss = train_model(ECAunet, train_loader, val_loader, criterion2, device, num_epochs=80)
metrics = evaluate_metrics(results_ECA_BCELoss)

[Epoch 1] Training: 100%|██████████| 45/45 [00:09<00:00,  4.62it/s]


[Epoch 1] Train Loss: 0.8501, Val Loss: 0.7949, IoU: 0.0373


[Epoch 2] Training: 100%|██████████| 45/45 [00:09<00:00,  4.62it/s]


[Epoch 2] Train Loss: 0.7615, Val Loss: 0.7337, IoU: 0.0203


[Epoch 3] Training: 100%|██████████| 45/45 [00:09<00:00,  4.63it/s]


...

[Epoch 47] Train Loss: 0.6926, Val Loss: 0.6932, IoU: 0.2488


[Epoch 48] Training: 100%|██████████| 45/45 [00:09<00:00,  4.66it/s]


[Epoch 48] Train Loss: 0.6925, Val Loss: 0.6944, IoU: 0.1841


[Epoch 49] Training: 100%|██████████| 45/45 [00:09<00:00,  4.68it/s]


[Epoch 49] Train Loss: 0.6925, Val Loss: 0.6957, IoU: 0.1343
Early stopping triggered.
Accuracy: 0.9769, Precision: 0.3267, Recall: 0.1857, F1 Score: 0.2368
Confusion Matrix:
[[1419256   10772]
 [  22922    5226]]


In [25]:
# ComboLoss
ECAunet = ECAUNet(in_channels=4, out_channels=1).to(device)
results_ECA_ComboLoss = train_model(ECAunet, train_loader, val_loader, criterion3, device, num_epochs=80)
metrics = evaluate_metrics(results_ECA_ComboLoss)

[Epoch 1] Training: 100%|██████████| 45/45 [00:09<00:00,  4.61it/s]


[Epoch 1] Train Loss: 0.9009, Val Loss: 0.8785, IoU: 0.0215


[Epoch 2] Training: 100%|██████████| 45/45 [00:09<00:00,  4.70it/s]


[Epoch 2] Train Loss: 0.8479, Val Loss: 0.8661, IoU: 0.0233


[Epoch 3] Training: 100%|██████████| 45/45 [00:09<00:00,  4.69it/s]


...

[Epoch 78] Train Loss: 0.8062, Val Loss: 0.8067, IoU: 0.3603


[Epoch 79] Training: 100%|██████████| 45/45 [00:09<00:00,  4.68it/s]


[Epoch 79] Train Loss: 0.8062, Val Loss: 0.8067, IoU: 0.3695


[Epoch 80] Training: 100%|██████████| 45/45 [00:09<00:00,  4.68it/s]


[Epoch 80] Train Loss: 0.8062, Val Loss: 0.8066, IoU: 0.3685
Accuracy: 0.9856, Precision: 0.7022, Recall: 0.4368, F1 Score: 0.5386
Confusion Matrix:
[[1424815    5213]
 [  15854   12294]]
